In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from scipy.stats import skew, kurtosis

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


from calcula_descritores import calculate_statistics
from calcula_metricas import calculate_metrics

In [ ]:
# Carrega o conjunto de dados
data = np.load('classes.npy')
df = pd.DataFrame(data, columns=[f'feat_{i}' for i in range(2560)] + ['label'])

# Define a reprodutibilidade
np.random.seed(0)

# Particionamento do conjunto de dados
X = df.iloc[:, :-1].values
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
# Criação dos modelos
models = []
mls_values = [1, 10, 20, 30]
for mls in mls_values:
    dt_model = DecisionTreeClassifier(min_samples_leaf=mls)
    models.append(('DecisionTree', dt_model))

NumNeighbors_values = [1, 3, 5, 7]
for num_neighbors in NumNeighbors_values:
    knn_model = KNeighborsClassifier(n_neighbors=num_neighbors)
    models.append(('KNeighbors', knn_model))

kernel_values = ['poly', 'rbf', 'linear']
for kernel in kernel_values:
    svc_model = SVC(kernel=kernel)
    models.append(('SVC', svc_model))

nb_model = GaussianNB()
models.append(('GaussianNB', nb_model))

ensemble_model = VotingClassifier(models)
stacked_model = StackingClassifier(estimators=models, final_estimator=DecisionTreeClassifier())

# Treinamento dos modelos
ensemble_model.fit(X_train, y_train)
stacked_model.fit(X_train, y_train)

# Avaliação dos modelos
y_pred_ensemble = ensemble_model.predict(X_test)
y_pred_stacked = stacked_model.predict(X_test)

# Matriz de confusão e métricas de desempenho para Ensemble
cm_ensemble = confusion_matrix(y_test, y_pred_ensemble)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
classification_report_ensemble = classification_report(y_test, y_pred_ensemble)

# Matriz de confusão e métricas de desempenho para Stacking
cm_stacked = confusion_matrix(y_test, y_pred_stacked)
accuracy_stacked = accuracy_score(y_test, y_pred_stacked)
classification_report_stacked = classification_report(y_test, y_pred_stacked)

# Exibe resultados
print("Resultados para o modelo de Ensemble:")
print("Matriz de Confusão:\n", cm_ensemble)
print("Acurácia:", accuracy_ensemble)
print("Relatório de Classificação:\n", classification_report_ensemble)

print("\nResultados para o modelo Stacking:")
print("Matriz de Confusão:\n", cm_stacked)
print("Acurácia:", accuracy_stacked)
print("Relatório de Classificação:\n", classification_report_stacked)
